In [6]:
import os
import sys
import numpy as np
import pandas as pd
import json
import random

In [29]:
!pwd

/data/sulixin/research/contest/qangaroo_v1.1


In [11]:
def my_load(filename):
    data = json.load(open(filename))
    print('{} has {} samples'.format(os.path.basename(filename), len(data)))
    return data

train_wiki = my_load('/data/sulixin/research/contest/qangaroo_v1.1/wikihop/train.json')
dev_wiki = my_load('/data/sulixin/research/contest/qangaroo_v1.1/wikihop/dev.json')
train_mask_wiki = my_load('/data/sulixin/research/contest/qangaroo_v1.1/wikihop/train.masked.json')
dev_mask_wiki = my_load('/data/sulixin/research/contest/qangaroo_v1.1/wikihop/dev.masked.json')

train.json has 43738 samples
dev.json has 5129 samples
train.masked.json has 43738 samples
dev.masked.json has 5129 samples


In [23]:
train_medhop = my_load('/data/sulixin/research/contest/qangaroo_v1.1/medhop/train.json')
dev_medhop = my_load('/data/sulixin/research/contest/qangaroo_v1.1/medhop/dev.json')
train_mask_medhop = my_load('/data/sulixin/research/contest/qangaroo_v1.1/medhop/train.masked.json')
dev_mask_medhop = my_load('/data/sulixin/research/contest/qangaroo_v1.1/medhop/dev.masked.json')

train.json has 1620 samples
dev.json has 342 samples
train.masked.json has 1620 samples
dev.masked.json has 342 samples


In [12]:
train_wiki[1]

{'id': 'WH_train_1',
 'query': 'languages_spoken_or_written john osteen',
 'answer': 'english',
 'candidates': ['english', 'greek', 'koine greek', 'nahuatl', 'spanish'],
 'supports': ['A Christian (or ) is a person who follows or adheres to Christianity, an Abrahamic, monotheistic religion based on the life and teachings of Jesus Christ. "Christian" derives from the Koine Greek word "Christós" (), a translation of the Biblical Hebrew term "mashiach".',
  'Lakewood Church is a nondenominational charismatic Christian megachurch located in Houston, Texas. It is the largest congregation in the United States, averaging about 52,000 attendees per week. The 16,800-seat Lakewood Church Central Campus, home to four English-language services and two Spanish-language services per week, is located at the former Compaq Center. Joel Osteen is the senior pastor of Lakewood Church with his wife, Victoria, who serves as co-pastor. Lakewood Church is a part of the Word of Faith movement.',
  'Mexico (, 

In [13]:
train_mask_wiki[1]

{'supports': ['Houston is the most populous city in Texas and the fourth - most populous city in the United States after Chicago , located in Southeast Texas near the Gulf of Mexico . With a census - estimated 2014 population of 2 . 239 million within an area of , it also is the largest city in the Southern United States , as well as the seat of Harris County . It is the principal city of Houston  The Woodlands  Sugar Land , which is the fifth - most populated metropolitan area in the United States of America .',
  'John Hillery Osteen ( August 21 , 1921 -- January 23 , 1999 ) was the founder and first pastor of Lakewood Church in Houston , Texas , from its beginnings in 1959 until his death in 1999 . His son Joel Osteen then succeeded him as pastor .',
  'Lakewood Church is a nondenominational charismatic Christian megachurch located in Houston , Texas . It is the largest congregation in the United States , averaging about 52 , 000 attendees per week . The 16 , 800 - seat Lakewood Chu

In [26]:
def word_len(text):
    return len(text.split())
    
def avg(li):
    return sum(li) * 1.0 / len(li)

def avg_word_len(text_li):
    return avg([word_len(text) for text in text_li])

def my_stat(data):
    rv = {'avg_context_len': [avg_word_len(d['supports']) for d in data],
         'context_cnt': [len(d['supports']) for d in data],\
         'query_len': [word_len(d['query'])for d in data],\
         'answer_len': [word_len(d['answer'])for d in data],\
         'candi_cnt': [len(d['candidates']) for d in data],\
         'rel_len': [d['query'].count('_')+1 for d in data],\
         'rel': [d['query'].split()[0] for d in data]}
    df = pd.DataFrame(rv)
    print(df['rel'].value_counts())
    return df.describe()

my_stat(train_wiki)

instance_of                                         4686
located_in_the_administrative_territorial_entity    4153
occupation                                          3185
place_of_birth                                      2517
record_label                                        2305
genre                                               2198
country_of_citizenship                              1511
parent_taxon                                        1382
place_of_death                                      1075
inception                                            964
date_of_birth                                        803
country                                              742
headquarters_location                                666
part_of                                              625
subclass_of                                          613
sport                                                594
member_of_political_party                            564
publisher                      

avg_context_len   context_cnt     query_len    answer_len  \
count     43738.000000  43738.000000  43738.000000  43738.000000   
mean         84.818946     13.674676      3.417212      1.790091   
std          22.900465      8.990930      1.293328      1.016945   
min          16.333333      3.000000      1.000000      1.000000   
25%          72.000000      7.000000      3.000000      1.000000   
50%          82.541958     11.000000      3.000000      2.000000   
75%          94.176471     18.000000      4.000000      2.000000   
max         460.250000     63.000000     16.000000     10.000000   

          candi_cnt       rel_len  
count  43738.000000  43738.000000  
mean      19.843980      2.398098  
std       16.289565      1.478878  
min        2.000000      1.000000  
25%        6.000000      1.000000  
50%       14.000000      2.000000  
75%       31.000000      3.000000  
max       79.000000      7.000000

In [22]:
my_stat(train_mask_wiki)

avg_context_len   context_cnt     query_len  answer_len     candi_cnt  \
count     43738.000000  43738.000000  43738.000000     43738.0  43738.000000   
mean         99.121330     13.674676      3.417212         1.0     19.896337   
std          25.620462      8.990930      1.293328         0.0     16.401517   
min          16.666667      3.000000      1.000000         1.0      2.000000   
25%          84.750000      7.000000      3.000000         1.0      6.000000   
50%          97.181818     11.000000      3.000000         1.0     14.000000   
75%         110.400000     18.000000      4.000000         1.0     31.000000   
max         490.250000     63.000000     16.000000         1.0     74.000000   

            rel_len  
count  43738.000000  
mean       2.398098  
std        1.478878  
min        1.000000  
25%        1.000000  
50%        2.000000  
75%        3.000000  
max        7.000000

In [25]:
train_medhop[1]

{'candidates': ['DB00294',
  'DB00313',
  'DB00588',
  'DB00755',
  'DB00783',
  'DB02546',
  'DB02901',
  'DB06822',
  'DB08910'],
 'answer': 'DB06822',
 'id': 'MH_train_1',
 'supports': ['Prolonged treatment with bicalutamide induces androgen receptor overexpression and androgen hypersensitivity . BACKGROUND : Various hormone refractory prostate cancer cell models have been established with androgen depletion and have helped to clarify the mechanism for the transition into androgen-depletion independent status . However , the mechanism of bicalutamide resistance remains unclear because few cell models have been generated . METHODS : We generated a bicalutamide-resistant subline , LNCaP- O43633 , from LNCaP after prolonged treatment with bicalutamide . Androgen and/or bicalutamide responsiveness for proliferation and prostate-specific antigen ( PSA ) secretion were examined in vitro and in vivo . DB00624 and dihydrotestosterone ( DB02901 ) levels in xenografted tumors were analyzed by

In [27]:
my_stat(train_medhop)

interacts_with    1620
Name: rel, dtype: int64


avg_context_len  context_cnt  query_len  answer_len    candi_cnt  \
count      1620.000000  1620.000000     1620.0      1620.0  1620.000000   
mean        250.594948    36.444444        2.0         1.0     8.911111   
std          10.732042    17.442892        0.0         0.0     0.651506   
min         212.931034     5.000000        2.0         1.0     2.000000   
25%         243.937500    21.000000        2.0         1.0     9.000000   
50%         250.976515    29.000000        2.0         1.0     9.000000   
75%         257.809524    57.000000        2.0         1.0     9.000000   
max         284.181818    64.000000        2.0         1.0     9.000000   

       rel_len  
count   1620.0  
mean       2.0  
std        0.0  
min        2.0  
25%        2.0  
50%        2.0  
75%        2.0  
max        2.0

# 总结
数据规模可以，问题重复多，答案其实不能真的被支撑